In [1]:
from utils import ScoreKeeper
from agent import Agent
from main import train_agent, plot_scores, watch_agent

# Train DDPG Agent

In [2]:
solved, scores = train_agent(
    max_t=1000,
    n_episodes=2000,
    checkpoint_every=100
)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Episode 23	 Score: 39.50Environment solved in -77 episodes!	Average Score: 30.11
Episode 24	 Score: 39.48Environment solved in -76 episodes!	Average Score: 30.50
Episode 25	 Score: 39.43Environment solved in -75 episodes!	Average Score: 30.86
Episode 26	 Score: 39.47Environment solved in -74 episodes!	Average Score: 31.19
Episode 27	 Score: 39.37Environment solved in -73 episodes!	Average Score: 31.49
Episode 28	 Score: 39.46Environment solved in -72 episodes!	Average Score: 31.78
Episode 29	 Score: 39.42Environment solved in -71 episodes!	Average Score: 32.04
Episode 30	 Score: 39.35Environment solved in -70 episodes!	Average Score: 32.29
Episode 31	 Score: 39.44Environment solved in -69 episodes!	Average Score: 32.52
Episode 32	 Score: 39.41Environment solved in -68 episodes!	Average Score: 32.73
Episode 33	 Score: 39.29Environment solved in -67 episodes!	Average Score: 32.93
Episode 34	 Score: 39.41Environment solved in -66 episodes!	Average Score: 33.12
Episode 35	 Score: 39.44Envi

KeyboardInterrupt: 

## Plot Scores

In [ ]:
plot_scores(scores)

# Watch Trained Agent

In [ ]:
if solved:
    watch_agent(
        max_t=1000,
        n_episodes=5
    )